In [1]:
import pandas as pd

In [4]:
orders = pd.read_csv('olist_orders_dataset.csv')

In [5]:
items = pd.read_csv('olist_order_items_dataset.csv')

In [6]:
products = pd.read_csv('olist_products_dataset.csv')
categories = pd.read_csv('product_category_name_translation.csv')
customers = pd.read_csv('olist_customers_dataset.csv')

In [7]:
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
geo = pd.read_csv('olist_geolocation_dataset.csv')

In [8]:
master_df = pd.merge(items,orders, on='order_id',how='inner')

In [9]:
master_df = pd.merge(master_df,payments,on='order_id',how='left')

In [10]:
master_df = pd.merge(master_df,reviews[['order_id','review_score']],on='order_id',how='left')

In [11]:
products_eng = pd.merge(products,categories,on='product_category_name',how='left')
master_df = pd.merge(master_df,products_eng,on = 'product_id',how='left')

In [12]:
master_df = pd.merge(master_df,customers[['customer_id','customer_city','customer_state']],on='customer_id',how='left')
master_df = pd.merge(master_df,sellers[['seller_id','seller_city','seller_state']],on='seller_id',how='left')


In [13]:
date_cols = ['order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']
for col in date_cols:
  master_df[col] = pd.to_datetime(master_df[col])

In [14]:
master_df['days_diff'] = (master_df['order_delivered_customer_date'] - master_df['order_estimated_delivery_date']).dt.days

In [16]:
from sqlalchemy import create_engine

In [19]:
DB_TYPE = 'postgresql'
DB_DRIVER = 'psycopg2'
DB_USER = 'postgres' # Default user
DB_PASS = 'Sayan123' # The password you set during installation
DB_HOST = 'localhost' 
DB_PORT = '5432'
DB_NAME = 'olist_db'

In [20]:
connection_string = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

In [21]:
files_to_upload = {
    'olist_customers_dataset.csv': 'customers',
    'olist_geolocation_dataset.csv': 'geolocation',
    'olist_order_items_dataset.csv': 'order_items',
    'olist_order_payments_dataset.csv': 'payments',
    'olist_order_reviews_dataset.csv': 'reviews',
    'olist_orders_dataset.csv': 'orders',
    'olist_products_dataset.csv': 'products',
    'olist_sellers_dataset.csv': 'sellers',
    'product_category_name_translation.csv': 'category_translation'
}

In [22]:
import os

In [24]:
for file_name, table_name in files_to_upload.items():
    if os.path.exists(file_name):
        print(f"--- Processing {table_name} ---")
        df = pd.read_csv(file_name)
        
        # Basic Cleaning: Remove duplicates
        df = df.drop_duplicates()
        
        # Upload to Postgres
        # 'replace' creates the table if it doesn't exist
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Successfully loaded {len(df)} rows into '{table_name}' table.")
    else:
        print(f"Warning: {file_name} not found in current directory.")

print("\nAll 9 datasets are now live in PostgreSQL!")

--- Processing customers ---
Successfully loaded 99441 rows into 'customers' table.
--- Processing geolocation ---
Successfully loaded 738332 rows into 'geolocation' table.
--- Processing order_items ---
Successfully loaded 112650 rows into 'order_items' table.
--- Processing payments ---
Successfully loaded 103886 rows into 'payments' table.
--- Processing reviews ---
Successfully loaded 99224 rows into 'reviews' table.
--- Processing orders ---
Successfully loaded 99441 rows into 'orders' table.
--- Processing products ---
Successfully loaded 32951 rows into 'products' table.
--- Processing sellers ---
Successfully loaded 3095 rows into 'sellers' table.
--- Processing category_translation ---
Successfully loaded 71 rows into 'category_translation' table.

All 9 datasets are now live in PostgreSQL!
